In [2]:
import os
os.chdir(r"C:\Users\fares\OneDrive\Desktop\Mémoire")
print("Répertoire actuel :", os.getcwd())


Répertoire actuel : C:\Users\fares\OneDrive\Desktop\Mémoire


In [6]:
import pandas as pd

# === 1. Chargement des fichiers ===
df_fibre = pd.read_csv("panel_fibre_it_2018_2024.csv")
df_chomage = pd.read_csv("chomage.csv")
df_pop = pd.read_csv("population.csv")
df_filo = pd.read_csv("filo_total_2018_2021.csv")
df_densite = pd.read_csv("densite.csv")
df_ent_long = pd.read_csv("df_ent_long_2018_2023.csv")

# === 2. Affichage des colonnes ===
print("📁 Fibre :")
print(df_fibre.columns.to_list(), "\n")

print("📁 Chômage :")
print(df_chomage.columns.to_list(), "\n")

print("📁 Population :")
print(df_pop.columns.to_list(), "\n")

print("📁 FiLoSoFi :")
print(df_filo.columns.to_list(), "\n")

print("📁 Densité :")
print(df_densite.columns.to_list(), "\n")

print("📁 Entreprise :")
print(df_ent_long.columns.to_list(), "\n")


📁 Fibre :
['INSEE_COM', 'Fibre_it', 'annee'] 

📁 Chômage :
['INSEE_COM', 'annee', 'nb_chomeurs'] 

📁 Population :
['INSEE_COM', 'annee_utilisation', 'population'] 

📁 FiLoSoFi :
['INSEE_COM', 'pop_fiscale', 'med_nivvie', 'part_imposés', 'pauvrete', 'part_salaires', 'part_chomage', 'part_pensions', 'part_patrimoine', 'part_social', 'part_impots', 'd1_nivvie', 'd9_nivvie', 'ratio_d9_d1', 'annee'] 

📁 Densité :
['INSEE_COM', 'LIBGEO', 'DENS', 'LIBDENS', 'PMUN21', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'zone_type'] 

📁 Entreprise :
['INSEE_COM', 'annee', 'nb_creations'] 



C:\Users\fares\AppData\Local\Temp\ipykernel_9240\2722968121.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_filo = pd.read_csv("filo_total_2018_2021.csv")


In [7]:
# Renommer proprement la colonne
df_pop = df_pop.rename(columns={"population": "population_recensee"})

# Vérification
print(df_pop.columns)

Index(['INSEE_COM', 'annee_utilisation', 'population_recensee'], dtype='object')


In [8]:
# Garder seulement les colonnes utiles
df_zone = df_densite[["INSEE_COM", "zone_type"]].copy()

# Renommer la zone
df_zone = df_zone.rename(columns={"zone_type": "zone_type_2021"})

# Vérification
print(df_zone.head())

  INSEE_COM zone_type_2021
0     01001          Rural
1     01002          Rural
2     01004         Urbain
3     01005          Rural
4     01006          Rural


In [12]:
df_fibre = df_fibre[df_fibre["annee"].between(2018, 2021)]
df_chomage = df_chomage[df_chomage["annee"].between(2018, 2021)]
df_pop = df_pop[df_pop["annee_utilisation"].between(2018, 2021)]
df_filo = df_filo[df_filo["annee"].between(2018, 2021)]
df_ent_long = df_ent_long[df_ent_long["annee"].between(2018, 2021)]

In [13]:
# 1. Renommer proprement
df_pop = df_pop.rename(columns={"annee_utilisation": "annee", "population": "population_recensee"})

# 2. Harmoniser les codes communes
for df in [df_fibre, df_chomage, df_pop, df_filo, df_ent_long, df_zone]:
    df["INSEE_COM"] = df["INSEE_COM"].astype(str).str.zfill(5)



In [41]:
# === 4. Fusion progressive ===
df_panel = df_fibre.merge(df_chomage, on=["INSEE_COM", "annee"], how="left")
df_panel = df_panel.merge(df_pop, on=["INSEE_COM", "annee"], how="left")
df_panel = df_panel.merge(df_filo, on=["INSEE_COM", "annee"], how="left")
df_panel = df_panel.merge(df_zone.rename(columns={"zone_type": "zone_type_2021"}), on="INSEE_COM", how="left")
df_panel = df_panel.merge(df_ent_long, on=["INSEE_COM", "annee"], how="left")

In [42]:
colonnes_exclues = ["INSEE_COM", "zone_type_2021"]

# conversion en numérique
for col in df_panel.columns:
    if col not in colonnes_exclues:
        df_panel[col] = pd.to_numeric(df_panel[col], errors="coerce")



In [43]:
# === 5. Création des indicateurs économiques ===
df_panel["taux_chomage_fiscal"] = (df_panel["nb_chomeurs"] / df_panel["pop_fiscale"]) * 100
df_panel["taux_chomage_INSEE"] = (df_panel["nb_chomeurs"] / df_panel["population_recensee"]) * 100
df_panel["tx_crea_ent_fiscale"] = (df_panel["nb_creations"] / df_panel["pop_fiscale"]) * 1000
df_panel["tx_crea_ent_recensee"] = (df_panel["nb_creations"] / df_panel["population_recensee"]) * 1000



In [44]:
df_panel.head()


,INSEE_COM,Fibre_it,annee,nb_chomeurs,population_recensee,pop_fiscale,med_nivvie,part_imposés,pauvrete,part_salaires,...,part_impots,d1_nivvie,d9_nivvie,ratio_d9_d1,zone_type_2021,nb_creations,taux_chomage_fiscal,taux_chomage_INSEE,tx_crea_ent_fiscale,tx_crea_ent_recensee
0,01001,0,2018,35.0,767.0,808.0,25350.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Rural,7.0,4.331683,4.563233,8.663366,9.126467
1,01001,0,2019,40.0,767.0,819.0,23970.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Rural,6.0,4.884005,5.215124,7.326007,7.822686
2,01001,0,2020,40.0,776.0,842.0,24820.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Rural,10.0,4.750594,5.154639,11.876485,12.886598
3,01001,0,2021,45.0,771.0,895.0,25820.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Rural,7.0,5.027933,5.836576,7.821229,9.079118
4,01002,1,2018,25.0,241.0,255.0,23890.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Rural,3.0,9.803922,10.373444,11.764706,12.448133


In [46]:
df_panel.to_excel("df_panel_2018_2021.xlsx", index=False)